In [41]:
from tensorflow import keras
import database_connect as db_connect
import pandas as pd
from datetime import timedelta

In [7]:
CONFIG = db_connect.load_config()

In [15]:
columns = ['id',
    'hour_sin',
 'hour_cos',
 'day_sin',
 'day_cos',
 'mon_sin',
 'mon_cos',
 'weekday_sin',
 'weekday_cos',
 'year',
 'RATIO_close_and_MA3',
 'RATIO_close_and_MA6',
 'RATIO_close_and_MA12',
 'RATIO_close_and_MA24',
 'RATIO_close_and_MA48',
 'RATIO_close_and_MA96',
 'RATIO_close_and_MA192',
 'RATIO_close_and_MA384',
 'RATIO_quote_asset_volume_and_MA3',
 'RATIO_quote_asset_volume_and_MA6',
 'RATIO_quote_asset_volume_and_MA12',
 'RATIO_quote_asset_volume_and_MA24',
 'RATIO_quote_asset_volume_and_MA48',
 'RATIO_quote_asset_volume_and_MA96',
 'RATIO_quote_asset_volume_and_MA192',
 'RATIO_quote_asset_volume_and_MA384',
 'RATIO_num_trades_and_MA3',
 'RATIO_num_trades_and_MA6',
 'RATIO_num_trades_and_MA12',
 'RATIO_num_trades_and_MA24',
 'RATIO_num_trades_and_MA48',
 'RATIO_num_trades_and_MA96',
 'RATIO_num_trades_and_MA192',
 'RATIO_num_trades_and_MA384',
 'RATIO_close_and_STD3',
 'RATIO_close_and_STD6',
 'RATIO_close_and_STD12',
 'RATIO_close_and_STD24',
 'RATIO_close_and_STD48',
 'RATIO_close_and_STD96',
 'RATIO_close_and_STD192',
 'RATIO_close_and_STD384',
 'RSI_3_close',
 'RSI_6_close',
 'RSI_12_close',
 'RSI_24_close',
 'RSI_48_close',
 'RSI_96_close',
 'RSI_192_close',
 'RSI_384_close', 
 'predicted']

In [4]:
model = keras.models.load_model('models/02-02-2024 22-12-58 val_PR 0.994 time frame 3.hd5', compile=False)

In [5]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 3, 49)]           0         
                                                                 
 normalization_1 (Normaliza  (None, 3, 49)             99        
 tion)                                                           
                                                                 
 lstm_2 (LSTM)               (None, 3, 49)             19404     
                                                                 
 lstm_3 (LSTM)               (None, 6)                 1344      
                                                                 
 dense_2 (Dense)             (None, 6)                 42        
                                                                 
 dense_3 (Dense)             (None, 4)                 28        
                                                           

In [63]:
conn = db_connect.connect(CONFIG)

Connected to the PostgreSQL server.


In [77]:
sql = 'select min(timestamp) from trading_bot_historydataeth \
        where predicted is null and hour_sin is not null \
        limit 1'

In [78]:
with conn.cursor() as cursor:
    cursor.execute(sql)
    result = cursor.fetchone()

In [80]:
result = result[0]

In [81]:
result_str = result.strftime('%Y-%m-%d %H:%M:%S')
result_str

'2024-01-09 07:00:00'

In [82]:
sql = f"SELECT * FROM trading_bot_historydataeth \
        WHERE timestamp >= '{result_str}' \
        ORDER BY timestamp;"

In [83]:
with conn.cursor() as cursor:
    cursor.execute(sql)
    result = cursor.fetchall()

In [84]:
result[0]

(38191,
 datetime.datetime(2024, 1, 9, 7, 0, tzinfo=datetime.timezone(datetime.timedelta(seconds=18000))),
 2302.0,
 2310.64,
 2298.58,
 2306.6,
 6331.7233,
 1704790000000,
 14591774.01,
 13559,
 0.9659258262890683,
 -0.25881904510252085,
 0.9510565162951536,
 -0.30901699437494734,
 0.49999999999999994,
 0.8660254037844387,
 0.7818314824680298,
 0.6234898018587336,
 2024,
 1.00035128957971,
 0.9989750104664423,
 0.995239038660473,
 1.0058934412616478,
 1.0210739071555812,
 1.0248298873913402,
 1.013136380956146,
 1.0079882211642606,
 0.7885732757532626,
 0.732956161611278,
 0.4653591645657934,
 0.310975557345064,
 0.38577078152515176,
 0.47220234804195693,
 0.37240879962283957,
 0.3907881626750921,
 0.732562537144092,
 0.6945853183750832,
 0.4871394910885101,
 0.3435158501440922,
 0.3870958532581543,
 0.46925158124888466,
 0.39762038223564933,
 0.4070589164262291,
 667.2412445236877,
 508.5179715216741,
 169.73353966464228,
 56.71755321926045,
 49.12727679528492,
 66.16669987138461,
 3

In [19]:
df_predict = pd.DataFrame(result, columns=columns)

ValueError: 51 columns passed, passed data had 60 columns

In [ ]:
model.predict()